<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/UH-Insure/Colab-Training.git"
REPO="Colab-Training"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

Cloning repo 'UH-Insure-NSA'...
Cloning into 'UH-Insure-NSA'...
remote: Enumerating objects: 413, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 413 (delta 106), reused 12 (delta 12), pack-reused 243 (from 2)
Receiving objects: 100% (413/413), 625.73 KiB | 14.90 MiB/s, done.
Resolving deltas: 100% (233/233), done.
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 73.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 33.8 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [2]:
from model.test import test
test()

Hello World!


In [3]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 20.4 MB/s eta 0:00:00
Logged in as: tam2003


In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
os.chdir("/content")
!pip install -q transformers datasets wandb peft bitsandbytes flash-attn


In [ ]:
from dataclasses import dataclass, field, asdict
from typing import List, Optional

@dataclass
class TrainingConfig:
    model_name: str = "Qwen/Qwen3-Coder-30B-A3B-Instruct"
    dataset_path: str = "/content/all_hybrid.jsonl"
    data_column: str = "content"
    seq_length: int = 4096
    overlap_ratio: float = 0.125
    max_steps: int = 200
    num_train_epochs: int = 1
    per_device_train_batch_size: int = 6
    per_device_eval_batch_size: int = 6
    gradient_accumulation_steps: int = 2
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03
    warmup_steps: int = 100
    weight_decay: float = 0.05
    lr_scheduler_type: str = "cosine"
    eval_frequency: int = 25
    save_frequency: int = 50
    log_frequency: int = 10
    output_dir: str = "peft-FT-3-Coder-30b-v2"
    bf16: bool = True
    fp16: bool = False
    # Set to "float16" to run the base model in fp16 precision.
    model_dtype: str = "bfloat16"
    fim_rate: float = 0.25
    fim_spm_rate: float = 0.5
    lora_r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    lora_target_modules: List[str] = field(default_factory=lambda: [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ])
    use_nested_quant: bool = True
    bnb_4bit_compute_dtype: str = "bfloat16"
    seed: int = 0
    wandb_project: str = "colab-training"
    wandb_entity: Optional[str] = None
    wandb_run_name: Optional[str] = None
    wandb_watch_models: bool = True
    overfit_patience: int = 2
    overfit_min_delta: float = 0.1
    max_unfrozen_layers: int = 2
    push_to_hub: bool = True

config = TrainingConfig()
print(config)


In [ ]:
import gc
import math
import os
import time
from typing import Dict, Optional

import torch
import wandb
from datasets import Dataset, concatenate_datasets, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainerCallback,
    TrainingArguments,
    set_seed,
)
from transformers.integrations import WandbCallback

from model.dataset.constantLengthDataset import ConstantLengthDataset


def chars_token_ratio(dataset: Dataset, tokenizer: AutoTokenizer, data_column: str, nb_examples: int = 400) -> float:
    """Estimate the average number of characters per token in the dataset."""
    total_characters, total_tokens = 0, 0
    iterator = iter(dataset)
    for _ in tqdm(range(nb_examples), total=nb_examples):
        try:
            example = next(iterator)
        except StopIteration:
            break
        text = example[data_column]
        total_characters += len(text)
        total_tokens += len(tokenizer(text).tokens())
    if total_tokens == 0:
        return 1.0
    return total_characters / total_tokens


class OverfittingDetector(TrainerCallback):
    def __init__(self, patience: int, min_delta: float, stage_name: str, run: Optional[object] = None):
        self.patience = patience
        self.min_delta = min_delta
        self.stage_name = stage_name
        self.run = run
        self.reset(stage_name)

    def reset(self, stage_name: Optional[str] = None):
        if stage_name is not None:
            self.stage_name = stage_name
        self.wait = 0
        self.training_losses = []
        self.eval_losses = []
        self.best_eval_loss = float("inf")
        self.best_training_loss = float("inf")
        self.last_training_loss = None
        self.overfit_detected = False

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        metrics: Dict[str, float] = {}
        if "loss" in logs:
            loss = logs["loss"]
            self.training_losses.append((state.global_step, loss))
            self.last_training_loss = loss
            if loss < self.best_training_loss - self.min_delta:
                self.best_training_loss = loss
                self.wait = 0
            metrics[f"{self.stage_name}/train_loss"] = loss
        if "learning_rate" in logs:
            metrics[f"{self.stage_name}/learning_rate"] = logs["learning_rate"]
        if "grad_norm" in logs:
            metrics[f"{self.stage_name}/grad_norm"] = logs["grad_norm"]
        if "tokens_per_second" in logs:
            metrics[f"{self.stage_name}/tokens_per_second"] = logs["tokens_per_second"]
        if metrics and self.run is not None:
            wandb.log(metrics, step=state.global_step)

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is None or "eval_loss" not in metrics:
            return
        eval_loss = metrics["eval_loss"]
        self.eval_losses.append((state.global_step, eval_loss))
        ppl = math.exp(eval_loss) if eval_loss < 20 else float("inf")
        log_metrics = {
            f"{self.stage_name}/eval_loss": eval_loss,
            f"{self.stage_name}/eval_perplexity": ppl,
        }
        if "eval_runtime" in metrics:
            log_metrics[f"{self.stage_name}/eval_runtime"] = metrics["eval_runtime"]
        if "eval_samples_per_second" in metrics:
            log_metrics[f"{self.stage_name}/eval_samples_per_second"] = metrics["eval_samples_per_second"]
        if self.run is not None:
            wandb.log(log_metrics, step=state.global_step)
        if eval_loss + self.min_delta < self.best_eval_loss:
            self.best_eval_loss = eval_loss
            self.wait = 0
        else:
            if self.last_training_loss is not None and eval_loss > self.last_training_loss + self.min_delta:
                self.wait += 1
            else:
                self.wait = max(self.wait - 1, 0)
        if self.wait >= self.patience:
            self.overfit_detected = True
            control.should_training_stop = True
            control.should_save = True
            control.should_evaluate = True


class StatisticsCallback(TrainerCallback):
    def __init__(self, stage_name: str, run: Optional[object] = None):
        self.stage_name = stage_name
        self.run = run
        self.start_time = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()

    def on_step_end(self, args, state, control, **kwargs):
        if self.run is None:
            return
        metrics = {f"{self.stage_name}/global_step": state.global_step}
        if torch.cuda.is_available():
            metrics.update(
                {
                    f"{self.stage_name}/gpu_mem_allocated_mb": torch.cuda.memory_allocated() / 1e6,
                    f"{self.stage_name}/gpu_mem_reserved_mb": torch.cuda.memory_reserved() / 1e6,
                }
            )
        if self.start_time is not None and state.global_step > 0:
            elapsed = time.time() - self.start_time
            metrics[f"{self.stage_name}/time_per_step_sec"] = elapsed / state.global_step
        wandb.log(metrics, step=state.global_step)


class EpochToDatasetCallback(TrainerCallback):
    def __init__(self, dataset: ConstantLengthDataset):
        self.dataset = dataset

    def on_epoch_begin(self, args, state, control, **kwargs):
        if hasattr(self.dataset, "set_epoch"):
            self.dataset.set_epoch(int(state.epoch) if state.epoch is not None else 0)


class TrainingPipeline:
    def __init__(self, cfg: TrainingConfig):
        self.cfg = cfg
        self.tokenizer = None
        self.train_dataset = None
        self.eval_dataset = None
        self.model = None
        self.trainer: Optional[Trainer] = None
        self.detector: Optional[OverfittingDetector] = None
        self.statistics_cb: Optional[StatisticsCallback] = None
        self.wandb_run = None
        self.stage_history = []
        self.global_best_eval_loss = float("inf")
        self.unfrozen_layers = 0

    def initialize(self):
        print("[INFO] Initializing training pipeline...")
        set_seed(self.cfg.seed)
        self._setup_wandb()
        self._prepare_tokenizer()
        self._prepare_datasets()
        self._prepare_model()
        if self.cfg.wandb_watch_models and self.wandb_run is not None:
            wandb.watch(self.model, log="all", log_freq=self.cfg.log_frequency)

    def _setup_wandb(self):
        mode = "online" if os.environ.get("WANDB_API_KEY") else "offline"
        wandb_kwargs = {
            "project": self.cfg.wandb_project,
            "config": asdict(self.cfg),
            "mode": mode,
        }
        if self.cfg.wandb_entity:
            wandb_kwargs["entity"] = self.cfg.wandb_entity
        if self.cfg.wandb_run_name:
            wandb_kwargs["name"] = self.cfg.wandb_run_name
        self.wandb_run = wandb.init(**wandb_kwargs)

    def _prepare_tokenizer(self):
        print("[INFO] Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.cfg.model_name,
            trust_remote_code=True,
            use_fast=False,
        )

    def _prepare_datasets(self):
        print("[INFO] Loading dataset and preparing splits...")
        dataset = load_dataset("json", data_files={"data": self.cfg.dataset_path})["data"]
        unsupervised = dataset.filter(lambda x: x.get("set") == "unsupervised")
        cryptol = unsupervised.filter(lambda x: x.get("filetype") == "cry")
        saw = unsupervised.filter(lambda x: x.get("filetype") == "saw")
        text = unsupervised.filter(lambda x: x.get("filetype") == "txt")
        cryptol_split = cryptol.train_test_split(test_size=0.1, seed=self.cfg.seed, shuffle=True)
        saw_split = saw.train_test_split(test_size=0.1, seed=self.cfg.seed, shuffle=True)
        text_split = text.train_test_split(test_size=0.1, seed=self.cfg.seed, shuffle=True)
        train_ds = concatenate_datasets([cryptol_split["train"], saw_split["train"], text_split["train"]])
        eval_ds = concatenate_datasets([cryptol_split["test"], saw_split["test"], text_split["test"]])
        assert self.cfg.data_column in train_ds.column_names, f"Missing '{self.cfg.data_column}' in training dataset"
        assert self.cfg.data_column in eval_ds.column_names, f"Missing '{self.cfg.data_column}' in eval dataset"
        cpt = chars_token_ratio(train_ds, self.tokenizer, self.cfg.data_column)
        print(f"[INFO] Character/token ratio: {cpt:.2f}")
        train_dataset = ConstantLengthDataset(
            tokenizer=self.tokenizer,
            dataset=train_ds,
            infinite=False,
            seq_length=self.cfg.seq_length,
            chars_per_token=cpt,
            content_field=self.cfg.data_column,
            fim_rate=self.cfg.fim_rate,
            fim_spm_rate=self.cfg.fim_spm_rate,
            overlap_ratio=self.cfg.overlap_ratio,
            seed=self.cfg.seed,
            already_tokenized=False,
        )
        eval_dataset = ConstantLengthDataset(
            tokenizer=self.tokenizer,
            dataset=eval_ds,
            infinite=False,
            seq_length=self.cfg.seq_length,
            chars_per_token=cpt,
            content_field=self.cfg.data_column,
            fim_rate=self.cfg.fim_rate,
            fim_spm_rate=self.cfg.fim_spm_rate,
            overlap_ratio=self.cfg.overlap_ratio,
            seed=self.cfg.seed,
            already_tokenized=False,
        )
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        if self.wandb_run is not None:
            wandb.log(
                {
                    "dataset/train_sequences": len(train_dataset),
                    "dataset/eval_sequences": len(eval_dataset),
                    "dataset/char_token_ratio": cpt,
                },
                step=0,
            )

    def _prepare_model(self):
        print("[INFO] Loading model and applying LoRA...")
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.set_float32_matmul_precision("high")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        model_dtype_name = self.cfg.model_dtype
        try:
            model_dtype = getattr(torch, model_dtype_name)
        except AttributeError as exc:
            raise ValueError(f"Unsupported model_dtype '{model_dtype_name}'") from exc
        compute_dtype_name = self.cfg.bnb_4bit_compute_dtype or model_dtype_name
        try:
            compute_dtype = getattr(torch, compute_dtype_name)
        except AttributeError as exc:
            raise ValueError(f"Unsupported bnb_4bit_compute_dtype '{compute_dtype_name}'") from exc
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=self.cfg.use_nested_quant,
        )
        print(f"[INFO] Model precision: model_dtype={model_dtype_name}, bnb_compute_dtype={compute_dtype_name}")
        if self.wandb_run is not None:
            wandb.log(
                {
                    "precision/model_dtype": model_dtype_name,
                    "precision/bnb_compute_dtype": compute_dtype_name,
                },
                step=0,
            )
        base_model = AutoModelForCausalLM.from_pretrained(
            self.cfg.model_name,
            quantization_config=bnb_config,
            dtype=model_dtype,
            device_map="auto",
            use_cache=True,
            trust_remote_code=True,
            attn_implementation="flash_attention_2",
        )
        base_model = prepare_model_for_kbit_training(base_model)
        peft_config = LoraConfig(
            r=self.cfg.lora_r,
            lora_alpha=self.cfg.lora_alpha,
            lora_dropout=self.cfg.lora_dropout,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=self.cfg.lora_target_modules,
        )
        self.model = get_peft_model(base_model, peft_config)
        self.model.print_trainable_parameters()

    def _create_trainer(self, stage_name: str):
        collator = DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False)
        training_args = TrainingArguments(
            output_dir=self.cfg.output_dir,
            max_steps=self.cfg.max_steps,
            num_train_epochs=self.cfg.num_train_epochs,
            per_device_train_batch_size=self.cfg.per_device_train_batch_size,
            per_device_eval_batch_size=self.cfg.per_device_eval_batch_size,
            gradient_accumulation_steps=self.cfg.gradient_accumulation_steps,
            gradient_checkpointing=False,
            learning_rate=self.cfg.learning_rate,
            lr_scheduler_type=self.cfg.lr_scheduler_type,
            warmup_steps=self.cfg.warmup_steps,
            warmup_ratio=self.cfg.warmup_ratio,
            weight_decay=self.cfg.weight_decay,
            fp16=self.cfg.fp16,
            bf16=self.cfg.bf16,
            optim="paged_adamw_8bit",
            logging_strategy="steps",
            logging_steps=self.cfg.log_frequency,
            evaluation_strategy="steps",
            eval_steps=self.cfg.eval_frequency,
            save_strategy="steps",
            save_steps=self.cfg.save_frequency,
            report_to=["wandb", "tensorboard"],
            push_to_hub=self.cfg.push_to_hub,
            dataloader_drop_last=False,
            dataloader_num_workers=2,
            include_tokens_per_second=True,
            max_grad_norm=0.3,
            save_total_limit=2,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            dataloader_pin_memory=True,
            save_safetensors=True,
            seed=self.cfg.seed,
            run_name=f"{stage_name}-run",
        )
        self.detector = OverfittingDetector(
            patience=self.cfg.overfit_patience,
            min_delta=self.cfg.overfit_min_delta,
            stage_name=stage_name,
            run=self.wandb_run,
        )
        self.statistics_cb = StatisticsCallback(stage_name=stage_name, run=self.wandb_run)
        callbacks = [self.detector, self.statistics_cb, EpochToDatasetCallback(self.train_dataset)]
        if self.wandb_run is not None:
            callbacks.append(WandbCallback())
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=collator,
            callbacks=callbacks,
        )
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.model.parameters())
        print(f"[INFO] Stage {stage_name}: trainable params {trainable_params:,} / {total_params:,}")
        if self.wandb_run is not None:
            wandb.log(
                {
                    "model/trainable_params": trainable_params,
                    "model/total_params": total_params,
                    "stage/name": stage_name,
                },
                step=0,
            )

    def train_stage(self, stage_name: str):
        print(f"[INFO] Starting training stage '{stage_name}'")
        self._create_trainer(stage_name)
        train_output = self.trainer.train()
        eval_metrics = self.trainer.evaluate()
        stage_record = {
            "stage": stage_name,
            "train_metrics": train_output.metrics,
            "eval_metrics": eval_metrics,
            "overfit_detected": self.detector.overfit_detected,
        }
        self.stage_history.append(stage_record)
        if self.wandb_run is not None:
            summary_metrics = {f"{stage_name}/{k}": v for k, v in eval_metrics.items() if isinstance(v, (int, float))}
            summary_metrics[f"{stage_name}/overfit_detected"] = float(self.detector.overfit_detected)
            wandb.log(summary_metrics, step=self.trainer.state.global_step)
        eval_loss = eval_metrics.get("eval_loss")
        degraded = False
        if eval_loss is not None:
            if eval_loss < self.global_best_eval_loss:
                self.global_best_eval_loss = eval_loss
            else:
                degraded = eval_loss > self.global_best_eval_loss + self.cfg.overfit_min_delta
        return {"metrics": eval_metrics, "overfit": self.detector.overfit_detected, "degraded": degraded}

    def push_to_hub(self, stage: str):
        if not self.cfg.push_to_hub:
            print("[INFO] push_to_hub disabled by configuration.")
            return
        commit_message = f"Stage {stage} checkpoint"
        print(f"[INFO] Pushing stage '{stage}' to the Hugging Face Hub...")
        self.trainer.push_to_hub(commit_message=commit_message)

    def unfreeze_next_layer(self) -> bool:
        if not hasattr(self.model, "base_model") or not hasattr(self.model.base_model, "model"):
            print("[WARN] Unable to locate base model layers for unfreezing.")
            return False
        layers = getattr(self.model.base_model.model, "layers", None)
        if layers is None:
            print("[WARN] Model does not expose layers attribute; cannot unfreeze further.")
            return False
        if self.unfrozen_layers >= min(self.cfg.max_unfrozen_layers, len(layers)):
            print("[INFO] Reached maximum number of unfrozen layers.")
            return False
        layer_index = len(layers) - 1 - self.unfrozen_layers
        for param in layers[layer_index].parameters():
            param.requires_grad = True
        self.unfrozen_layers += 1
        print(f"[INFO] Unfroze encoder layer index {layer_index} ({self.unfrozen_layers}/{self.cfg.max_unfrozen_layers}).")
        if self.wandb_run is not None:
            wandb.log(
                {
                    "model/unfrozen_layer_index": layer_index,
                    "model/total_unfrozen_layers": self.unfrozen_layers,
                }
            )
        return True

    def finish(self):
        if self.wandb_run is not None:
            wandb.finish()
        print("[INFO] Training pipeline complete.")


In [ ]:
pipeline = TrainingPipeline(config)
try:
    pipeline.initialize()
    base_stage = pipeline.train_stage("lora_base")
    if base_stage["overfit"]:
        pipeline.push_to_hub(stage="lora_base")
    for _ in range(config.max_unfrozen_layers):
        if not pipeline.unfreeze_next_layer():
            break
        stage_name = f"unfreeze_layer_{pipeline.unfrozen_layers}"
        stage_result = pipeline.train_stage(stage_name)
        if stage_result["overfit"]:
            pipeline.push_to_hub(stage=stage_name)
        if stage_result["degraded"]:
            print(f"[WARN] Degraded performance detected after stage '{stage_name}'. Stopping further fine-tuning.")
            break
finally:
    pipeline.finish()
    print("Stage history:")
    for record in pipeline.stage_history:
        print(record)
